In [10]:
import sys

sys.path.append("/home/ezetap/Documents/personal/works/interviews")

In [11]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.ensemble import IsolationForest
import numpy as np

In [13]:
df = pd.read_csv("/home/ezetap/Documents/personal/works/interviews/data/historical_records.csv")
df

,Date,Product,Price,Quantity
0,2021-01-01,Apple,1.25,10.0
1,2021-01-02,Orange,NaN,5.0
2,2021-01-03,Banana,0.75,20.0
3,2021-01-04,Apple,1.15,15.0
4,2021-01-05,Banana,0.80,18.0
5,2021-01-06,Orange,1.05,8.0
6,2021-01-07,NaN,0.90,12.0
7,2021-01-08,Apple,1.20,NaN
8,2021-01-09,Orange,1.10,25.0
9,2021-01-10,Banana,0.85,30.0


In [14]:
imputer = SimpleImputer(strategy='most_frequent')
data_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
data_imputed

,Date,Product,Price,Quantity
0,2021-01-01,Apple,1.25,10.0
1,2021-01-02,Orange,0.75,5.0
2,2021-01-03,Banana,0.75,20.0
3,2021-01-04,Apple,1.15,15.0
4,2021-01-05,Banana,0.8,18.0
5,2021-01-06,Orange,1.05,8.0
6,2021-01-07,Apple,0.9,12.0
7,2021-01-08,Apple,1.2,5.0
8,2021-01-09,Orange,1.1,25.0
9,2021-01-10,Banana,0.85,30.0


In [16]:
# Identify high variance records using Isolation Forest
isolation_forest = IsolationForest(contamination=0.1)  # Adjust the contamination parameter as needed
isolation_forest.fit(data_imputed[["Price", "Quantity"]])

outliers = isolation_forest.predict(data_imputed[["Price", "Quantity"]])
high_variance_records = data_imputed[["Price", "Quantity"]][outliers == -1]
high_variance_records

/home/ezetap/.cache/pypoetry/virtualenvs/interviews-cHlQfc8P-py3.8/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


,Price,Quantity
9,0.85,30.0


In [18]:
# Calculate the variance of numerical columns
variance = df.select_dtypes(include='number').var()
variance

Price        0.034028
Quantity    66.861111
dtype: float64

In [19]:
# Set a threshold to identify high variance records
threshold = 10  # Adjust the threshold as needed

# Filter records with variance above the threshold
high_variance_records = df[df.columns[variance > threshold]]
high_variance_records

IndexError: boolean index did not match indexed array along dimension 0; dimension is 4 but corresponding boolean dimension is 2

In [20]:
df.select_dtypes(include='number').var(axis=1, skipna=True)

0     38.28125
1          NaN
2    185.28125
3     95.91125
4    147.92000
5     24.15125
6     61.60500
7          NaN
8    285.60500
9    424.86125
dtype: float64

In [17]:
outliers

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1, -1])

In [ ]:
# Identify and impute null values
imputer = SimpleImputer(strategy='most_frequent')
data_imputed = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

# Identify high variance records using Isolation Forest
isolation_forest = IsolationForest(contamination=0.1)  # Adjust the contamination parameter as needed
isolation_forest.fit(data_imputed)

outliers = isolation_forest.predict(data_imputed)
high_variance_records = data_imputed[outliers == -1]

# Optional: Replace high variance records with imputed values
imputed_high_variance_records = pd.DataFrame(imputer.transform(high_variance_records), columns=data.columns)
data_imputed.update(imputed_high_variance_records)

# Optional: Save the cleansed data to a new CSV file
data_imputed.to_csv('cleansed_records.csv', index=False)